## **Access DB2 on Cloud using Python**

**Objectives:**

Establishing a connection to a database instance of DB2 Warehouse on Cloud from a Python notebook using ibm_db API following the below tasks:

* Create a table
* Insert data into the table
* Query data from the table
* Retrieve the result set into pandas dataframe
* Close the dabase connection
    
    
    

#### 1: Import the `ibm_db` Python library

In [ ]:
!pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
!pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
!pip install ipython-sql

In [ ]:
import ibm_db

#### 2: Identify the database connection credentials

In [ ]:
#Replace the placeholder values with the actuals for your Db2 Service Credentials
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "database"            # e.g. "BLUDB"
dsn_hostname = "hostname"            # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_port = "port"                    # e.g. "50000" 
dsn_protocol = "protocol"            # i.e. "TCPIP"
dsn_uid = "username"                 # e.g. "abc12345"
dsn_pwd = "password"                 # e.g. "7dBZ3wWt9XN6$o0J"
dsn_security = "SSL"              #i.e. "SSL"

#### 3: Create the database connection

In [ ]:
#Create database connection

dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )


#### 4: Create a table in the database

In [ ]:
#Lets first drop the table INSTRUCTOR in case it exists from a previous attempt
dropQuery = "drop table INSTRUCTOR"

#Now execute the drop statment
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

Getting an error `Exception: [IBM][CLI Driver][DB2/LINUXX8664] SQL0204N "ABC12345.INSTRUCTOR" is an undefined name. SQLSTATE=42704 SQLCODE=-204` is normal if the table does not exist

In [ ]:
#Construct the Create Table DDL statement - replace the ... with rest of the statement
createQuery = "create table INSTRUCTOR(id INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20). LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2))"

#Now fill in the name of the method and execute the statement
createStmt = ibm_db.replace_with_name_of_execution_method(conn, createQuery)

#### 5: Insert data into the table

In [ ]:
#Construct the query - replace ... with the insert statement
insertQuery = "insert into INSTRUCTOR values(1, 'Rav', 'Ahuja', 'Toronto', 'CA')"

#execute the insert statement
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

In [ ]:
#replace ... with the insert statement that inerts the remaining two rows of data
insertQuery2 = "insert into INSTRUCTOR values (2, 'Raul', 'Chong', 'Markham', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US')"

#execute the statement
insertStmt2 = ibm_db.exec_immediate(conn, insertQuery2)

#### 6: Query data in the table

In [ ]:
#Construct the query that retrieves all rows from the INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#Fetch the Dictionary (for the first row only) - replace ... with your code
ibm_db.getch_both(selectStmt)

In [ ]:
#Fetch the rest of the rows and print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
   print (" ID:",  ibm_db.result(selectStmt, 0), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))

In [ ]:
# Change City for Rav to Moosetown

UpdateQuery = "update INSTRUCTOR ser CITY = 'Moosetown whare FName = 'Rav'"
UpdateSTMT = ibm_db.exec.immeciate(conn, UpdateQuery)

#### 7: Retrieve data into Pandas

In [14]:
import pandas
import ibm_db_dbi

In [ ]:
# connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [ ]:
# query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

# retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

# print just the LNAME for first row in the pandas data frame
pdf.LNAME[0]

In [15]:
# Print the entire data frame
pdf

NameError: name 'pdf' is not defined